In [1]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 57.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 49.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [3]:
import mlflow.sklearn
from datetime import datetime
import joblib
import dagshub
import mlflow
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm import tqdm

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=18c09f31-155f-4f12-9b6b-2db54f8a5f76&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=f3c0c7ff394546cdc3e8a74d0f3f14609e40b575d0e2e89e2bd7830d28fce37e




Output()

Accessing as gnada22

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

In [4]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["weekofyear"] = X["Date"].dt.isocalendar().week.astype(int)
        X["sin_13"] = np.sin(2 * np.pi * X["weekofyear"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["weekofyear"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["weekofyear"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["weekofyear"] / 23)
        X = X.drop(columns=["Date"])
        return X

local_path = mlflow.artifacts.download_artifacts(
    artifact_uri="mlflow-artifacts:/bdaee488c065473c96d054e5ae43539a/abb40eeefeb54f7a87763486cb01f016/artifacts/model.pkl"
)

model = joblib.load(local_path)

print("Done!")

Done!


In [6]:
# load and add lag features

test = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip", parse_dates=["Date"])
features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = test.merge(features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(stores, on="Store", how="left")

def add_lag_features(df):
    train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
    train = train[["Store", "Dept", "Date", "Weekly_Sales"]]
    train = train.sort_values(["Store", "Dept", "Date"])
    
    full = pd.concat([train, df], axis=0)
    full = full.sort_values(["Store", "Dept", "Date"])
    
    full["lag_1"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
    full["lag_52"] = full.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)

    res = full[full["Weekly_Sales"].isna()].copy() # just test rows
    # res.dropna(subset=["lag_1", "lag_52"], inplace=True)  # drop rows with missing lags

    return res

df = add_lag_features(df)

X_test = df.drop(columns=["Weekly_Sales"], errors="ignore")

print("Done!")

Done!


In [28]:
X_test = X_test.sort_values(["Date", "Store", "Dept"]).reset_index(drop=True)
X_test["Weekly_Sales_Predicted"] = np.nan

# Create a lookup table for easier lag value access
pred_lookup = {}

# Iterate by day
for date in tqdm(X_test["Date"].sort_values().unique(), desc="Recursive prediction"):
    day_rows = X_test[X_test["Date"] == date].copy()

    for idx, row in day_rows.iterrows():
        store = row["Store"]
        dept = row["Dept"]

        # Get keys for previous lags
        key_1 = (store, dept, date - pd.Timedelta(weeks=1))
        key_52 = (store, dept, date - pd.Timedelta(weeks=52))

        lag_1 = pred_lookup.get(key_1, row["lag_1"])
        lag_52 = pred_lookup.get(key_52, row["lag_52"])

        # print(store, dept, date, lag_1)

        day_rows.at[idx, "lag_1"] = lag_1
        day_rows.at[idx, "lag_52"] = lag_52

    # Predict all rows for this day in one batch
    y_preds = model.predict(day_rows)

    # Assign predictions back
    X_test.loc[day_rows.index, "Weekly_Sales_Predicted"] = y_preds
    # print(X_test.loc[day_rows.index, ["Weekly_Sales_Predicted", "Date", "Store", "Dept"]])
    # print(y_preds)

    # Update lookup for future lag access
    for idx, pred in zip(day_rows.index, y_preds):
        row = day_rows.loc[idx]
        key = (row["Store"], row["Dept"], row["Date"])
        pred_lookup[key] = pred

# X_test = X_test.sort_values(["Store", "Dept", "Date"])
# print(X_test[["Store", "Dept", "Date", "Weekly_Sales_Predicted"]])

submission["Weekly_Sales"] = X_test["Weekly_Sales_Predicted"]
submission["Id"] = X_test["Store"].astype(str) + "_" + X_test["Dept"].astype(str) + "_" + X_test["Date"].dt.strftime("%Y-%m-%d")
# print(submission)
submission.to_csv("submission.csv", index=False)
print("✅ Submission saved as submission.csv")

Recursive prediction: 100%|██████████| 39/39 [00:27<00:00,  1.44it/s]


✅ Submission saved as submission.csv
